In [ ]:
import tensorflow as tf

## Model (모델)

텐서들을 가지고 연산 및 예측을 수행하는 객체를 `model`이라고 한다.  모델은 두가지의 일을 수행한다.

- 텐서간 연산을 통한 결과값 도출 (forward pass)
- 학습을 통한 가중치 갱신 (backward pass)


딥러닝에서의 모델은 데이터를 처리하는 과정에서 여러개의 layer들로 구성되기 때문에 모델을 구현하기 전에 layer를 구현하는 것이 먼저 선행되어야한다

# Layer

재사용이 가능하며, 훈련이 가능한 변수들로 이루어진 수학적 구조를 말한다.   
단순히 입력에 대한 처리(연산)를 진행하고 그 결과를 출력하기 때문에 하나의 함수로 볼 수 있다.   
layer를 함수로 표현하게 되면 하나의 모델은 함수들의 연속, 합성함수로 볼 수 있게 될 것이다.

<img src="./education_images/1-1-1_formula.png" alt="Drawing" style="width: 400px;"/>

<img src="./education_images/1-1-2_graph.png" alt="Drawing" style="width: 300px;"/>

tensorflow에서는  tf.Module 의 클래스를 상속받아 새로운 클래스를 생성하는 방식으로 layer를 구현한다.   

-  callable 객체이기 때문에 `__call__`을 내부적으로 구현해야한다.

https://docs.python.org/3/library/functions.html

- 이후 사용할 keras API 에서의 layer, model 또한 해당 객체의 하위 클래스이다.


가장 단순한 인공신경망 layer인 fully connected layer를 tensorflow로 구현해보자.   
코드로 구현하기 전, layer는 하나의 함수이기 때문에 입력과 출력의 형태와 어떠한 __연산__을 수행할 것 인가에 대해서 정의가 필요하다.


<img src="./education_images/fullylayer.png" alt="Drawing" style="width: 400px;"/>

## 1. 입력과 출력의 shape 정의

layer의 입력, 출력 모두 텐서이다. (연산의 과정까지 모두 텐서이다.)   
텐서는 모두 shape, dtype이 정의되어야한다.
> 맨 앞 차원의 크기가 None인 것은 batch의 크기 때문이다.

<img src="./education_images/1-1-3_input.png" alt="Drawing" style="width: 250px;"/>


## 2. 연산식 정의


<img src="./education_images/1-1-4_formula.png" alt="Drawing" style="width: 300px;"/>

## 3. 텐서플로우 구현

클래스 내에서 두가지의 함수를 정의하여 사용한다. 각 함수의 첫 번째 특징은 해당 함수를 정의해야하는 이유다.


### 3-1. `__init__`



- python에서 클래스를 정의할 때, 해당 클래스의 객체 생성시 호출되는 함수이다.  




- `super(self).__init__()` 을 통해서 상위클래스인 `tf.Module`객체를  생성한다.   
    → 상위 클래스 객체를 생성할 때, 상위클래스의 property, method를 사용할 수 있다.
    
    
    
- 클래스를 통해서 layer 객체를 생성하는 시점에 해당 함수가 호출된다.  


- 해당 layer에서 사용할 텐서 객체(변수, 상수, layer, 모델)등 을 선언하는 영역이다.(필수는 아니다.)  
    → Variable은 반드시 초기값이 필요한데, 모델을 생성하는 시점에 반드시 명시해줘야한다.




### 3-2. `__call__`

-  `tf.Module` 가 callable 객체이기 때문에 해당 함수를 오버라이딩 해야한다.  
    → 하위 클래스가 상위 클래스의 method를 재정의하여 사용하는 것을 오버라이딩이라고 한다.


- layer 객체를 호출하여 데이터를 입력받는 시점에 해당 함수가 호출된다. 


- 이 전에 `__init__` 에서 미리 선언해둔 텐서 객체들을 사용하여 수행할 연산을 정의한다. 

## 4. 우리가 만든 Layer 클래스로 layer 객체 생성 및 수행


- `trainable_variables`는 `tf.Module` 의 학습가능한 정의된 변수들에 반환한다.   
    이 때, 정의된 변수들이라는 의미는 초기값과 shape가 정해진 상태의 변수들을 의미한다.
    
    
- layer 객체를 생성하는 시점에 변수를 정의하는 방식( `__init__` 내에서 변수들을 정의하는 방식)를 사용하게 되면 해당 layer의 input의 shape가 반드시 명시되어야 한다. 이러한 명시는 해당 layer 객체에 대한 shape의 제약을 주게되고 동일한 연산임에도 input의 shape가 달라질 때마다 새로운 객체를 생성해야만 한다. input의 shape에 대한 제약을 줄일 수 없을까?

## 5. 우리가 만든 Layer 클래스로 객체생성 시점에 shape 정의하기

`__init__`내부에서 사용할 텐서 객체를 생성하는 것이 아니라, `__call__` 에서 텐서 객체를 생성하게 되면 사용할 변수의 정의가 layer 객체의 호출시점에 발생한다. 즉, layer 객체 호출시 전달되는 입력값의 shape에 따라 변수의 shape를 정의할 수 있게 된다. 아래의 예제를 보자.

- 클래스를 통해서 layer 객체를 생성하고난 뒤  `trainable_variables` 의 값을 살펴보면 어떠한 변수도 존재하지 않음을 알 수 있다. 객체 호출 시점에 변수가 정의되기 때문에 존재하지 않는 것이다.  


- `is_built` 변수 값을 통해서 해당 모델 객체가 여러번 호출되더라도 내부의 텐서 객체가 재생성되는 것을 막는다.  
 

- 해당 layer에서 사용하는 변수 w의 shape를 입력값의 shape를 반영하여 사용한다.   
    → 행렬의 곱셈이 성립하기 위해서는 아래의 공식을 만족해야하기 때문에 x의 가장 마지막 차원의 크기을 맞추어준다. 
    
    
<img src="./education_images/1-1-5_formula.png" alt="Drawing" style="width: 400px;"/>


- 최초로 layer 객체가 호출되는 시점에 내부 변수 W의 shape가 정해지게 되고, 최초 입력의 가장 마지막 차원의 크기가 동일한 입력에 대해서는 shape가 다르더라도 layer 객체를 사용할 수 있게 된다.

## Model 만들어보기

- 모델 또한 `tf.Module` 을 상속받은 하위클래스이다. 


- `submodules` 을 사용하여 모델 내부에서 사용된 `tf.Module`의 하위클래스로 생성된 layer, model 객체를 확인할 수 있다.



- `print(model.variables)` → 코드의 결과를 출력하기 전에 각각의 shape를 계산해보자!


간단한 모델을 tensorflow를 사용하여 구축해보았다. 가장 기본적인 모델인 선형회귀 모델과 학습 및 가중치 조정을 tensorflow로 구현하여 머신러닝의 전과정을 구현해보도록 하자.

# 머신러닝의 문제 해결 과정

머신러닝의 문제해결 과정은 이 전에 배운 것과는 다를 수 있다. 이번 실습은 아래의 절차에 따라 진행된다.

1. 해결할 문제에 대해 정의한다.


2. 학습 데이터를 정의하고 수집한다.


3. 문제 해결을 위한 모델을 정의한다.

4. 손실 함수 (loss function)를 정의한다.    


5. 학습 데이터로 모델의 예측값을 구하고 정의된 loss function을 통해서 loss를 구한다. (forward pass)


6. loss에 대한 gradient를 계산하고 optimizer를 통해서 모델의 가중치를 조정한다.


7. 5,6번의 과정을 학습루프라고 정의하는데, 학습루프를 반복하고 최종적으로 결과를 평가한다.
  

# 1. 문제정의

> 아래의 수식을 만족하는 데이터에 적합한 선형 회귀 모델을 구하라. 
   
<img src="./education_images/1-1-6_formula.png" alt="Drawing" style="width: 500px;"/>

# 2. 학습 데이터 수집

# 3. 모델 정의

- $\hat{y} \ =\  w * x \  +\  b$ 의 식을 tensorflow로 구현하였다. 


- 이후 미분값을 통한 가중치 계산시 반드시 변수들은 float type 이어야 한다.


- 초기값을 w, b 각각 10, 0으로 주었다.

# 4. 손실 함수 정의

손실 함수란  $y - \hat{y}$, 실제 값과 모델이 예측한 값의 차이이다.
  
<img src="./education_images/1-1-7_formula.png" alt="Drawing" style="width: 300px;"/>

# 5. 학습 루프 정의

학습 과정은 아래와 같다.

1. 모델을 통해 각 batch 마다 예측값 $\hat{y}$ 구한다.


2. 사전에 정의해둔 loss function을 통해 loss를 구한다.


3. 자동미분을 수행하는 GradientTape를 통해 gradient를 구한다.


4. optimizer를 통해 가중치를 갱신한다.

# 6. 학습 루프를 진행하며 결과 평가

### `batch`

1회의 학습루프에 사용할 학습 데이터의 크기


### `epoch`

전체 학습 데이터에 대해서 진행할 학습의 횟수를 의미한다.


- 위에서 epoch는 30으로, 총 30번의 학습 루프를 진행하였다. 또한 학습 루프 내에서 학습 데이터로 batch로 분할하여 예측값을 구하고 가중치를 갱신하는데, 이 과정을 1회만 수행하였기 때문에 1 batch 내에 학습데이터를 모두 사용한 것이다. 즉 batch는 1000이 된다. 


- 학습루프를 진행하면서 변수로 정의되어있는 w, b 의 값이 각각 3, 1로 수렴하고 있음을 볼 수 있으며 loss 또한 지속적으로 낮아지는 것을 확인할 수 있다. w, b 값의 변화를 plot을 통해 시각화하여보고, 우리의 회귀선이 데이터에 적합한지도 시각화하여보자. 

# 가중치 저장하기

지금까지 tensorflow로 구현한 모든 layer, model이 `tf.Module` 의  하위클래스로 구현됨을 보았다. 우리가 학습된 모델을 저장하고 싶을 때, 학습과정마다 모델을 저장하고자 할 때 `tf.train.Checkpoint()`를 사용하여 가중치를 저장한다. 저장하는 방법은 아래와 같다.


1. `checkpoint` 객체를 생성하여 현재 모델의 상태를 저장한다.


2. `checkpoint.save`를 통해서 디렉토리에 저장한다.
    -  저장되는 객체는  
        `<model명>-<save counter 수>.data-00000-of-00001`  
        →  변수 값과 실제 데이터
        
       `<model명>-<save counter 수>.index`  
        →  저장된 항목과 체크포인트의 번호를 추적하는 메타 데이터
        
- 코드가 실행되는 디렉토리의 하위디렉토리로 가중치 및 데이터들이 저장된다.

- 새로운 모델을 생성하게 되면 가중치의 값이 모델을 선언했을 때의 초기값을 가진다.

- 새로운 모델에 대해서 미리 저장된 가중치를 적용하기 위해서는 새로운 모델에 대해서 _checkpoint_ 를 생성한 뒤, 저장된 가중치를 `restore` 를 통해서 불러온다. 

- 이외에 모델 객체를 저장하는 방법이 존재하지만 대부분의 레퍼런스, 기존의 모델들의 경우, keras를 통해 사용되기 때문에 keras API에서 다루려고 한다.